### importing libraries

In [1]:
import pandas as pd
import json

In [2]:
data_adj = pd.read_csv('adj.csv')
data_adv = pd.read_csv('adv.csv')
data_noun = pd.read_csv('noun.csv')
data_verb = pd.read_csv('verb.csv')

In [3]:
def eval_csv(data):
    pbi_words_synonym = []
    for i in range (0,len(data)):
        try :
            pbi_words_synonym.append(data['punjabi'][i].split())
        except:
            pbi_words_synonym.append('NaN')

    eng2pun = {}
    row,_ = data.shape
    for i in range(row-1):
        eng2pun[data['english'].iloc[i]] = pbi_words_synonym[i]
    return eng2pun

In [4]:
def removing_multiphrase(my_dict):
    ## removing multiphase punjabi words::
    for eng,pbi in my_dict.items():
        for i in pbi:
            if i.find("_") != -1:
                pbi.remove(i)
    return my_dict

### here we will change our files that is data_adj, data_adv, data_noun and data_verb

In [26]:
## evaluating csv to dict
noun_dict = eval_csv(data_noun)
noun_dict = removing_multiphrase(noun_dict)

In [14]:
## importing english and punjbai words count from our dataset
f = open('eng.json')
eng_count = json.load(f)

f = open('punjabi.json')
pbi_count = json.load(f)

In [5]:
def filtering_top1000(data_dict,eng_count):
    ## making count of words from our dataset
    file_count={}
    for input in data_dict:
        for key,value in eng_count.items():
            if key == input:
                file_count[key]=value

    #descending order
    counts = dict(sorted(file_count.items(), key=lambda x:x[1],reverse=True))

    #filtering top 1000
    my_dict = {}
    cnt = 0 
    for key,value in counts.items():
        if cnt >= 1000:
            break
        my_dict[key]=value
        cnt+=1

    # pushing the filtered dict 
    filtered_dict={}
    for inp,value in data_dict.items():
        if inp in my_dict:
            filtered_dict[inp]=value
    return filtered_dict


In [6]:
def filtering_pbi_words(filtered_dict,pbi_count):
    temp_2={}
    #now filtering the punjabi words by criterias that the most occured word from the list will be taken 
    # counting the frequency of each punjabi word (from our dataset) and considering the maximum one.
    for key,pbi_list in filtered_dict.items():
        maxi = 0
        for i in pbi_list:
            if i in pbi_count.keys():
                maxi = max(maxi, pbi_count[i])
        ## this for loop will return the frequency of maximum word occured out of all
        for i in pbi_list:
            if i in pbi_count.keys():
                pbi_count[i] == maxi
                temp_2[key] = i
        ## this loop will find the word with max frequency and pushing back to new dict
    return temp_2

In [27]:
temp = filtering_top1000(noun_dict,eng_count)
temp = filtering_pbi_words(temp,pbi_count)

In [28]:
len(temp)

227

In [29]:
with open('noun_final.json', 'w') as f:
    json.dump(temp, f)

In [ ]:
# for eng,pbi_list in temp.items():
#     count = 0
#     pbi_count_dict={}
#     for i in range (0,len(pbi_list)):
#         if pbi_list[i] in pbi_count:
#             pbi_count_dict[pbi_list[i]] = pbi_count[pbi_list[i]]
#         else:
#             pbi_count_dict[pbi_list[i]]=0
    